In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalAveragePooling1D, DepthwiseConv2D, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

IMAGE_SIZE = 96

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir,'train+val','train')
     valid_path = os.path.join(base_dir,'train+val','valid')


     RESCALING_FACTOR = 1./255

     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

os.chdir('..')
dir=os.getcwd()


c:\Users\20222787\.conda\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
# et the filepath where the trained model is saved (should be in the 'metadata' directory with a .keras extension)
model_name='double_CNN_1'
model_filepath = dir+'/metadata/'+model_name + '_weights.keras'

# load the model from the given filepath
model=tf.keras.models.load_model(model_filepath)

train_gen, val_gen = get_pcam_generators(dir+'\Data')

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
steps = val_gen.samples // val_gen.batch_size  
loss, accuracy, area_under_curve, recall = model.evaluate(val_gen, steps=steps)

print(f'loss: {loss:.4f}')
print(f'accuracy: {accuracy:.4f}')
print(f'AUC: {area_under_curve:.4f}')
print(f'Recall: {recall:.4f}')

500/500 [==============================] - 148s 280ms/step - loss: 0.3737 - accuracy: 0.8399 - auc_2: 0.9214 - recall_2: 0.8121
loss: 0.3737
accuracy: 0.8399
AUC: 0.9214
Recall: 0.8121


In [ ]:
y_pred_prob = list(model.predict(val_gen).flatten())  # Returns a probability per image
y_pred=[1 if num >= .5 else 0 for num in y_pred_prob]
y_true = list(val_gen.classes)  # The real labels (0 or 1) from the validation set

accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary') 

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

 451/4500 [==>...........................] - ETA: 13:29

KeyboardInterrupt: 

In [38]:
# Initialize counters
num_batches = 500
predictions = []
true = []

# Iterate through the first 20 batches of the data generator
for i, (batch_data, batch_labels) in enumerate(val_gen):
    if i >= num_batches:
        break  # Stop after 20 batches
    
    # Perform prediction on the batch
    batch_predictions = model.predict(batch_data)  # You can also use model(batch_data, training=False)
    
    # Store predictions for later analysis (optional)
    predictions.append(batch_predictions)
    true.append(batch_labels)

# After the loop, you can convert the predictions list into a single array (optional)
predictions = np.concatenate(predictions, axis=0)
true = np.concatenate(true, axis=0)

1/1 [==============================] - 0s 50ms/step


In [ ]:
correct_predictions = 0
total_samples = 0

# Iterate through the generator
for batch_data, batch_labels in val_gen:
    predictions = model.predict(batch_data)
    correct_predictions += np.sum(np.argmax(predictions, axis=-1) == np.argmax(batch_labels, axis=-1))
    total_samples += batch_data.shape[0]

accuracy = correct_predictions / total_samples

TypeError: '>=' not supported between instances of 'slice' and 'int'

In [24]:
batch_data, batch_labels = enumerate(val_gen)
#predictions=model.predict(val_gen.data)

ValueError: too many values to unpack (expected 2)

In [39]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(true, predictions)

In [41]:
roc_auc = auc(fpr, tpr)
roc_auc


0.9214428576464306

In [7]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    
# Compute the AUC
roc_auc = auc(fpr, tpr)
print(f'area under curve: {roc_auc:.4f}')

from sklearn.metrics import confusion_matrix as cm
confusion_matrix=cm(y_pred,y_true)
print(confusion_matrix)


area under curve: 0.4987
[[4219 4225]
 [3781 3775]]


In [15]:
y_pred_prob

[0.7921773,
 0.3115063,
 0.9832361,
 0.5951114,
 0.73207295,
 0.51762617,
 0.563621,
 0.43391788,
 0.0096657835,
 0.8208519,
 0.7215363,
 0.97667664,
 0.025579551,
 0.22818902,
 0.042315755,
 0.037798014,
 0.5644913,
 0.8168321,
 0.6083153,
 0.82833314,
 0.842327,
 0.11371771,
 0.11500127,
 0.97037804,
 0.19034566,
 0.31499138,
 0.1122832,
 0.76609653,
 0.6875551,
 0.14121749,
 0.3826993,
 0.5190788,
 0.42129472,
 0.09194593,
 0.6321239,
 0.8966649,
 0.7873881,
 0.014704426,
 0.6344121,
 0.45554912,
 0.77950454,
 0.60650265,
 0.9878221,
 0.34395394,
 0.3477565,
 0.3021876,
 0.10596876,
 0.16137855,
 0.27302727,
 0.69876534,
 0.3111097,
 0.48570177,
 0.69822526,
 0.08216229,
 0.6902751,
 0.20450665,
 0.015153106,
 0.050185185,
 0.8384885,
 0.8439078,
 0.3299557,
 0.13387163,
 0.57882965,
 0.52197134,
 0.09521651,
 0.28900045,
 0.34653977,
 0.7088574,
 0.22532967,
 0.98155624,
 0.9986545,
 0.056945495,
 0.231327,
 0.017554505,
 0.5069082,
 0.32403713,
 0.8583547,
 0.1911538,
 0.5767172,


In [14]:
y_pred

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
